In [334]:
from quantum_logical._lib import apply_operators_in_place as rust_apply
from quantum_logical.error_channel import AmplitudeDamping, PhaseDamping
import numpy as np
import matplotlib.pyplot as plt
from qutip.metrics import fidelity
from qutip.operators import sigmax, sigmay, sigmaz
from qutip import basis
from qutip import expect
from quantum_logical.error_channel import ErrorChannel
from qutip import Qobj, tensor
from scipy.linalg import fractional_matrix_power

In [335]:
from quantum_logical.interaction import ConversionGainInteraction

gc, gg = np.pi / 2, 0
trotter_dt = 0.1
duration = 1.0

In [336]:
one_rho = basis(2, 1) * basis(2, 1).dag()  # |1><1|
initial_state = tensor([one_rho] * 2)


# |01><01|
initial_state = (
    tensor([basis(2, 0), basis(2, 1)]) * tensor([basis(2, 0), basis(2, 1)]).dag()
)

# bell state |00> + |11>
initial_state = tensor([basis(2, 0), basis(2, 0)]) + tensor([basis(2, 1), basis(2, 1)])
initial_state = initial_state * initial_state.dag()

# |00> + |01>
initial_state = tensor([basis(2, 0), basis(2, 0)]) + tensor([basis(2, 0), basis(2, 1)])
initial_state = initial_state * initial_state.dag()


initial_state /= initial_state.norm()
initial_state = initial_state.full()
initial_state = Qobj(initial_state)

In [337]:
H = ConversionGainInteraction(gc, gg)
iswap = H.construct_U(t=duration)
iswap

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

In [348]:
class MultiQubitErrorChannel(ErrorChannel):
    """A class for error channels that can be applied to multi-qubit systems."""

    def __init__(self, chi, trotter_dt):
        """
        Initializes with given arrays of T1 and T2 times for each qubit, and a trotter step size.
        """
        self.N = 2
        self.chi = chi
        self.num_qubits = 2
        dims = 2**self.num_qubits
        self.trotter_dt = trotter_dt
        super().__init__(trotter_dt, dims)

    def _init_kraus_operators(self):
        """
        Creates the multi-qubit Kraus operators for the error channels.
        """
        zz = tensor(sigmaz(), sigmaz())
        return (-1j * self.chi * zz * np.pi / 2 * self.trotter_dt).expm()


noise_channel = MultiQubitErrorChannel(0, trotter_dt)
new_state = noise_channel.apply_error_channel(initial_state, 1.0, unitary=iswap.full())
new_state

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[2441406.5       +9.60357713e-12j 2430220.66960511+9.47506831e+04j
  2430220.66960511+9.47511831e+04j 2441406.        +9.60357713e-12j]
 [2430220.66960511-9.47506831e+04j 2441406.        +1.02876332e-11j
  2441406.        +2.85089583e-10j 2430220.66960511-9.47506831e+04j]
 [2430220.66960511-9.47511831e+04j 2441406.        -2.51666123e-10j
  2441406.5       +1.20071995e-11j 2430220.66960511-9.47506831e+04j]
 [2441406.        +9.60357713e-12j 2430220.66960511+9.47506831e+04j
  2430220.66960511+9.47506831e+04j 2441406.        +9.60357713e-12j]]